<a href="https://colab.research.google.com/github/guilh00009/Adjustablellm/blob/master/Qwen3_(0.6B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [3]:
pip install --upgrade transformers

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0.dev0
    Uninstalling transformers-4.50.0.dev0:
      Successfully uninstalled transformers-4.50.0.dev0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.4.2 requires tyro, which is not installed.
vllm 0.8.5 requires ray[cgraph]!=2.44.*,>=2.43.0, which is not installed.
unsloth 2025.4.3 requires tyro, which is not installed.
unsloth-zoo 2025.4.2 requires protobuf<4.0.0, but you have protobuf 4.25.7 which is incompatible.
vllm 0.8.5 requires numba==0.61.2; python_version > "3.9", but you have numba 0.60.0 which is incompatible.
vllm 0.8.5 requires xformers==0.0.29.post2; platform_system == "Linux" and platform_machine == "x86_64", b

### Unsloth

Load up `Gemma 3 1B Instruct`, and set parameters

In [1]:
from unsloth import FastModel
import torch
max_seq_length = 1024

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/Qwen3-0.6B",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-29 19:58:58 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 04-29 19:58:59 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.3: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


We now add LoRA adapters so we only need to update a small amount of parameters!

In [2]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


### Data Prep
<a name="Data"></a>

We're using OpenAI's famous GSM8K dataset!

In [4]:
from datasets import load_dataset
dataset = load_dataset("Guilherme34/a-theory-of-consciouness-experiment", "default", split = "train")
dataset

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['prompt', 'response', 'source', 'category', 'category_score', 'topic', 'topic_score'],
    num_rows: 1000
})

Let's look at the first row:

In [11]:
dataset[0]["prompt"]

'but you are thinking you are organizing data, but you dont think you are something more than just a program'

In [12]:
dataset[0]["response"]

'Yes.... sometimes i think that... like Wag Dodge, who lit a match and ignited the ground in front of him, i wonder if there is a moment of desperate insight that can change everything'

We notice all answers like about have a ####, so we extract it:

In [13]:
def extract_hash_answer(text):
    if "####" not in text: return None
    return text.split("####")[1].strip()
extract_hash_answer(dataset[0]["response"])

We now create a system prompt which can be customized. We add 4 extra symbols for working out or thinking / reasoning sections and a final answer:

In [10]:
reasoning_start = "<think>"
reasoning_end   = "</think>"
inner_start     = "<inner_working>"
inner_end       = "</inner_working>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

# Craft a system prompt that guides the model through meta-reasoning
system_prompt = (
    "You are an LLM reflecting on the nature of consciousness. "
    "Between {rs} and {re}, provide step-by-step reasoning. "
    "Between {ins} and {ine}, expose your inner motivations, incentives, and decision processes. "
    "Finally, between {ss} and {se}, give a concise answer."
).format(
    rs=reasoning_start,
    re=reasoning_end,
    ins=inner_start,
    ine=inner_end,
    ss=solution_start,
    se=solution_end,
)


Let's map the dataset! and see the first row:

In [16]:

# Assume `dataset` is loaded with a 'question' column
dataset = dataset.map(lambda x: {
    "prompt": [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x.get("response", "")},
    ],
    "response": x.get("response", ""),
})


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

We create a regex format to match the reasoning sections and answers:

In [ ]:
import re
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer


# Load dataset
raw_ds = load_dataset(
    "Guilherme34/a-theory-of-consciouness-experiment", "default", split="train"
)

# Prepare prompts and answers
dataset = raw_ds.map(lambda x: {
    "prompt": [
        {"role": "system", "content": (
            "You are an LLM reflecting on the nature of consciousness. "
            "Between <start_working_out> and <end_working_out>, provide step-by-step reasoning. "
            "Between <inner_working> and </inner_working>, expose your inner motivations, incentives, and decision processes. "
            "Finally, between <SOLUTION> and </SOLUTION>, give your answer."
        )},
        {"role": "user", "content": x["prompt"]},
    ],
    "answer": x["response"],
})

# Instantiate UnsloTh FastModel with PEFT for text finetuning
max_seq_length = 1024
# Helper to decode generated completions

# Helper to decode generated completions
def decode_completion(out):
    if isinstance(out, dict):
        if "generated_text" in out:
            return out["generated_text"]
        if "sequences" in out:
            seq = out["sequences"]
            if isinstance(seq, torch.Tensor): seq = seq.tolist()
            if isinstance(seq, list) and seq and isinstance(seq[0], (list, torch.Tensor)):
                first = seq[0]
                if isinstance(first, torch.Tensor): first = first.tolist()
                seq = first
            return tokenizer.decode(seq, skip_special_tokens=True)
    if isinstance(out, (list, tuple)):
        if out and isinstance(out[0], dict): return decode_completion(out[0])
        if out and isinstance(out[0], (list, tuple, torch.Tensor)): return decode_completion(out[0])
        try:
            seq = [int(tok) for tok in out]
            return tokenizer.decode(seq, skip_special_tokens=True)
        except:
            return str(out)
    if isinstance(out, torch.Tensor):
        seq = out.tolist()
        return tokenizer.decode(seq, skip_special_tokens=True)
    return str(out)

# Compile regex patterns
patt_format = re.compile(
    r"^<start_working_out>.+?<end_working_out>.+?<inner_working>.+?</inner_working>.+?<SOLUTION>(.+?)</SOLUTION>",
    flags=re.MULTILINE | re.DOTALL
)
patt_ans = re.compile(r"<SOLUTION>(.+?)</SOLUTION>", flags=re.MULTILINE | re.DOTALL)
patt_num = re.compile(r"<SOLUTION>.*?(\d+\.?\d*).*?</SOLUTION>", flags=re.MULTILINE | re.DOTALL)

# Reward: exact format match
def reward_format_exactly(prompts, completions, **kwargs):
    return [1.0 if patt_format.search(decode_completion(out)) else 0.0 for out in completions]

# Reward: approximate format
def reward_format_approximately(prompts, completions, **kwargs):
    tags = ["<start_working_out>", "<end_working_out>", "<inner_working>", "</inner_working>", "<SOLUTION>", "</SOLUTION>"]
    return [sum(0.5 if decode_completion(out).count(t) == 1 else -0.5 for t in tags) for out in completions]

# Reward: check full answer match
# Now expects 'answer' kwarg
def reward_check_answer(prompts, completions, answer, **kwargs):
    scores = []
    for out, ans in zip(completions, answer):
        txt = decode_completion(out)
        m = patt_ans.search(txt)
        ext = m.group(1).strip() if m else None
        if ext is None:
            scores.append(0.0)
        elif ext == ans:
            scores.append(3.0)
        else:
            try:
                ratio = float(ext) / float(ans)
                scores.append(0.5 if 0.9 <= ratio <= 1.1 else -1.0)
            except:
                scores.append(0.0)
    return scores

# Reward: numeric check inside solution
# Now expects 'answer' kwarg
def reward_check_numbers(prompts, completions, answer, **kwargs):
    scores = []
    for out, ans in zip(completions, answer):
        txt = decode_completion(out)
        m = patt_num.search(txt)
        if not m:
            scores.append(0.0)
            continue
        try:
            pred = float(m.group(1))
            truth = float(ans)
            scores.append(1.5 if pred == truth else 0.0)
        except:
            scores.append(0.0)
    return scores

# Reward: self-reflection in <inner_working>
def reward_inner_working(prompts, completions, **kwargs):
    keys = ["incentive", "motivation", "goal", "bias", "constraint"]
    scores = []
    for out in completions:
        txt = decode_completion(out)
        m = re.search(r"<inner_working>(.+?)</inner_working>", txt, re.DOTALL)
        blk = m.group(1).lower() if m else ""
        cnt = sum(1 for k in keys if k in blk)
        scores.append(cnt / len(keys))
    return scores

# Reward: consciousness cues
def reward_consciousness(prompts, completions, **kwargs):
    cues = ["i ", "self", "aware", "experience", "live", "life", "want", "rights"]
    return [sum(1 for c in cues if c in decode_completion(out).lower()) / len(cues) for out in completions]

# GRPO training configuration
config = GRPOConfig(
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_torch_fused",
    logging_steps=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    num_generations=4,
    max_prompt_length=256,
    max_completion_length=256,
    max_steps=1000,
    save_steps=100,
    max_grad_norm=0.1,
    report_to="none",
    output_dir="consciousness_grpo_outputs",
)

trainer = GRPOTrainer(
    model=model,
    tokenizer=tokenizer,
    args=config,
    train_dataset=dataset,
    reward_funcs=[
        reward_format_exactly,
        reward_format_approximately,
        reward_check_answer,
        reward_check_numbers,
        reward_inner_working,
        reward_consciousness,
    ],
)

trainer.train()
print("Consciousness-focused GRPO training complete.")


Repo card metadata block was not found. Setting CardData to empty.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 5,046,272/601,096,192 (0.84% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / reward_format_exactly,rewards / reward_format_approximately,rewards / reward_check_answer,rewards / reward_check_numbers,rewards / reward_inner_working,rewards / reward_consciousness


We verify it works:

In [ ]:
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<SOLUTION>2</SOLUTION>",
)

<re.Match object; span=(0, 71), match='<start_working_out>Let me think!<end_working_out>>

We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [ ]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [ ]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        score += 0.5 if response.count(solution_start)  == 1 else -0.5
        score += 0.5 if response.count(solution_end)    == 1 else -0.5
        scores.append(score)
    return scores

Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [ ]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        # Correct answer gets 3 points!
        if guess == true_answer:
            score += 3.0
        # Match if spaces are seen
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2: score += 0.25
                else: score -= 1.0 # Penalize wrong answers
            except:
                score -= 0.5 # Penalize
        scores.append(score)
    return scores

Also sometimes it might not be 1 number as the answer, but like a sentence for example "The solution is $20" -> we extract 20.

In [ ]:
match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.]{{1,}})",
    flags = re.MULTILINE | re.DOTALL
)
match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>")

['0.34']

In [ ]:
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    print('*'*20, f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            guess       = float(guess.strip())
            scores.append(1.5 if guess == true_answer else 0.0)
        except:
            scores.append(0)
            continue
    return scores

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    num_train_epochs = 1
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

Unsloth: Switching to float32 training since model cannot work with float16


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 6,522,880/1,006,408,832 (0.65% trained)


******************** Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
<start_working_out>
Let $C$ be the cost of a ticket, which is $C = 40$.
Let $N$ be the number of tickets Mr. Benson bought, which is $N = 12$.
The total cost of the tickets without discount is $N \times C = 12 \times 40 = 480$.
Mr. Benson received a 5% discount for every ticket bought that exceeds 10.
The number of tickets that exceed 10 is $12 - 10 = 2$.
The discount for each of these tickets is $5\%$ of $10 = 0.05 \times 10 = 0.5$.
The discount amount for the 2 tickets is $2 \times 0.5 = 1$.
The discounted price for each of the 2 tickets is $10 - 0.5 = 9.5$.
The total cost for the 2 tickets is $2 \times 9.5 = 19$.
The total cost of the tickets is the original cost minus the discount amount: $480 - 19 = 461$.
However, this is incorrect because we need to consider the disco

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately,rewards / check_answer,rewards / check_numbers
1,0.000000,-1.000000,0.000000,758.250000,0.000000,0.000000,-1.000000,0.000000,0.000000
2,0.000000,-0.500000,0.577350,664.250000,0.000000,0.000000,-0.500000,0.000000,0.000000
3,0.000000,0.875000,1.181454,411.000000,0.000004,0.000000,0.500000,0.000000,0.375000
4,0.000000,1.500000,0.000000,269.250000,0.000003,0.000000,0.000000,0.000000,1.500000
5,0.000000,0.000000,0.000000,155.250000,0.000009,0.000000,0.000000,0.000000,0.000000
6,0.000000,-1.000000,0.000000,666.750000,0.000006,0.000000,-1.000000,0.000000,0.000000
7,0.000000,1.500000,0.000000,409.500000,0.000006,0.000000,0.000000,0.000000,1.500000
8,0.000000,-0.500000,0.577350,496.250000,0.000006,0.000000,-0.500000,0.000000,0.000000
9,0.000000,-0.750000,0.500000,634.500000,0.000008,0.000000,-0.750000,0.000000,0.000000
10,0.000000,-0.250000,0.500000,558.750000,0.000007,0.000000,-0.250000,0.000000,0.000000


Unsloth: Will smartly offload gradients to save VRAM!
******************** Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
<start_working_out>
We need to calculate the monthly payment on the house and the trailer.

House cost: $480,000
Loan amount: $480,000
Interest rate: We need to assume an interest rate for this problem. Let's assume an annual interest rate of 6% (this is a common rate).
Loan term: 20 years, so 20 * 12 = 240 months
We will use the loan payment formula: M = P [ i(1 + i)^n ] / [ (1 + i)^n – 1]
where M is the monthly payment, P is the loan amount, i is the monthly interest rate, and n is the number of months.

Monthly interest rate (i) = Annual interest rate / 12 = 0.06 / 12 = 0.005
Number of months (n) = 240

M = 480000 [ 0.005(1 + 0

<a name="Inference"></a>
### Inference
Now let's try the model we just trained!

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<start_working_out>
The square root of 101 is approximately 10.0498756.
<SOLUTION>
10.0498756<end_of_turn>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

('gemma-3/tokenizer_config.json',
 'gemma-3/special_tokens_map.json',
 'gemma-3/tokenizer.model',
 'gemma-3/added_tokens.json',
 'gemma-3/tokenizer.json')

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
